# 使い方

- 上から順にShift + Enterしていく。
- main部分のwordsに検索ワードのリストを入力してShift + Enter(入力するワード数に制限はない)


以上。

- このnotebookと同じディレクトリに"検索ワード1+検索ワード2+検索ワード3.csv"ができる。


## 注意


### chromedriver_binaryのバージョンについて
↓と思ったけど、以前会ったときできていたのでこれは多分大丈夫。

ターミナルから、
```
pip install chromedriver_binary
または
conda install chromedriver_binary
```
しただけだと動かなかった。
自分のchromeのバージョンが古かった。77。
よって自分の環境では
```
pip uninstall chromedriver_binary してから
pip install chromedriver_binary==77.0.3865.40

または

conda uninstall chromedriver_binary してから
conda install chromedriver_binary==77.0.3865.40
```
とする。
バージョンに指定する数値は

[スクレイピング時に「This version of ChromeDriver only supports Chrome version \.\.\.」のエラーが出た時 \- クラインの備忘壺](https://kleinblog.net/scraping-chrome-error/)

[Downloads \- ChromeDriver \- WebDriver for Chrome](http://chromedriver.chromium.org/downloads)

を参照。
 
 
 ### 取得数を増やしたいとき
 
 
 main部分の
 
 ```
 max_data_count = 2000
```

を増やす。



### 取得する属性を増やしたいとき

相談してください。

## 要件定義

欲しい情報

検索結果一覧のから各商品のページに移って(これを詳細ページと呼ぼう)、詳細ページの

右側にある

- 出品者の3段階評価マーク(顔のマーク)のそれぞれの個数
   ex.ニッコリマーク59個、真顔マーク1個、残念マーク0個
- 「商品の状態」欄のテキスト情報
   ex.新品、未使用
- 「配送料の負担」欄のテキスト情報
   ex.送料込み(出品者負担)
- 「配送の方法」欄のテキスト情報
   ex.らくらくメルカリ便
- 「配送元地域」欄のテキスト情報 
   ex.神奈川県

・値段
・販売中か売り切れたかのダミー
・左側にある掲載画像の枚数(写真自体ではなく枚数)
・値段の下にある商品説明のテキストデータ

これらのデータが欲しい。そして前回みたく検索ワードを変更出来る仕様、csvに落とし込んでくれる仕様


## 確認すべきこと

- 1件につき5秒、一覧一ページにつき5びょうかかるので1ページ100商品だと8分20秒かかる。 一覧ページは無尽蔵にあるようなので、100ページあるとすると13時間くらいかかる 。
- あまりに途方もなさすぎるので、取得数を限定する。


## その他開発にあたりメモ
    
    
テストすべきこと

-　成功 詳細一ページにつき各属性を要件通りに取得できるか→print
- 　成功　一覧一ページにつきdataframeをエラーなく作成できるか→isnull.max
- 成功　途中でブラウザが落ちないか
   

In [2]:
# ① ライブラリ読み込み部分
import requests
from bs4 import BeautifulSoup
import re
import os
import time
from time import sleep
import pandas as pd
from datetime import datetime
import traceback
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary

In [3]:
# soupから指定したセレクタを抜き出すが、存在しなかったらnullを返す関数

def select_from_soup(selector,soup):
    if soup.select(selector):
        target = soup.select(selector)[0].text
        return target
    else :
        return None
    

In [4]:
# ② 詳細ページから分析に必要な属性を抜き出して一覧ページに向けて返す関数

def get_detail_dictionary(link):
    driver.get(link)
    time.sleep(2) # ここをもっと少ない値にするともっと処理時間を短縮できるが、2秒は待ったほうが良い気がする。
    soup = BeautifulSoup(driver.page_source, "html.parser")        
    
    #ここから属性取得を開始する

    # 出品者の3段階評価マーク(顔のマーク)のそれぞれの個数   ex.ニッコリマーク59個、真顔マーク1個、残念マーク0個
#     good    = soup.select(".icon-good+span")[0].text 
    good = select_from_soup(".icon-good+span",soup)
    if good : 
        good    = int(good)                                                   #文字列型では分析できないので数値型に
#     normal =  soup.select(".icon-normal+span")[0].text
    normal = select_from_soup(".icon-normal+span",soup)
    if normal:
        normal = int(normal)
#     bad      = soup.select(".icon-bad+span")[0].text
    bad = select_from_soup(".icon-bad+span",soup)
    if bad:
        bad      = int(bad)

    # 「商品の状態」欄のテキスト情報   ex.新品、未使用
    # 4番目のtrに入っている
#     status  = soup.select(".item-detail-table tr:nth-child(4) td")[0].text
    status = select_from_soup(".item-detail-table tr:nth-child(4) td",soup)
    #    - 「配送料の負担」欄のテキスト情報   ex.送料込み(出品者負担)
    # 5番目のtrに入っている
#     charge  = soup.select(".item-detail-table tr:nth-child(5) td")[0].text
    charge = select_from_soup(".item-detail-table tr:nth-child(5) td",soup)

    #- 「配送の方法」欄のテキスト情報   ex.らくらくメルカリ便
    # ６番目
#     method = soup.select(".item-detail-table tr:nth-child(6) td")[0].text
    method = select_from_soup(".item-detail-table tr:nth-child(6) td",soup)

    # - 「配送元地域」欄のテキスト情報    ex.神奈川県
    # 7
#     region   = soup.select(".item-detail-table tr:nth-child(7) td")[0].text
    region = select_from_soup(".item-detail-table tr:nth-child(7) td",soup)

    # 値段
#     price = soup.select(".item-price.bold")[0].text.replace("¥","").replace(",","")
    price = select_from_soup(".item-price.bold",soup)
    if price: 
        price = price.replace("¥","").replace(",","")
        price = int(price)

    #販売中か売り切れたかのダミー
    #売り切れ商品のページには<div class="item-sold-out-badge"><div>SOLD</div></div>がある
    # 売り切れていたら1, 残っていたら0
    if soup.select(".item-sold-out-badge"):
        sold = 1
    else:
        sold = 0

    #   ・左側にある掲載画像の枚数(写真自体ではなく枚数)
    # 写真一枚は<div class="owl-dot-inner"> に入っている
    if soup.select(".owl-dot-inner"):
        image_count = len(soup.select(".owl-dot-inner"))
    else:
        image_count = None
    
    #・値段の下にある商品説明のテキストデータ
    # .item-description-inner　
#     description = soup.select(".item-description-inner")[0].text
    description = select_from_soup(".item-description-inner",soup)


    print(good,normal, bad, status, charge, method, region, price, sold, image_count, description[0:30])
#     print(normal)
#     print(bad)
#     print(status)
#     print(charge)
#     print(method)
#     print(region)
#     print(price)
#     print(sold)
#     print(image_count)
#     print(description[0:30])

    #データ名と値の辞書。詳細属性を追加したいときには上でスクレイピングしてここに追記すること。
    data_dictionary=  {"good": good,
           "normal":normal,
            "bad": bad,
            "status" : status,
            "charge": charge,
            "method": method, 
            "region": region,
            "price": price,
            "sold": sold, 
            "image_count": image_count,
           "description": description
#                        "timedelta": timedelta  　←追記の例
           }
    return data_dictionary


In [5]:
# ③検索ワードから一覧URLを生成する関数


def create_index_url(words):
    if (len(words) < 1):
        raise "検索ワードがありません。何か入力してください"
    words = list(map(lambda x : urllib.parse.quote(x), words))
    base_url = "https://www.mercari.com/jp/search/?keyword="
    return base_url + "%20".join(words)

# テスト
# 出力値がブラウザで直接検索したURLと完全に同一か： 同一なら　True
# 　成功
# create_index_url(["airpods","第二世代", "新品" ]) == "https://www.mercari.com/jp/search/?keyword=airpods+%E7%AC%AC%E4%BA%8C%E4%B8%96%E4%BB%A3+%E6%96%B0%E5%93%81"




In [6]:
# ●メイン部分




print("取得を開始します")
# url = "https://www.mercari.com/jp/search/?keyword=airpods+%E7%AC%AC2%E4%B8%96%E4%BB%A3+%E6%96%B0%E5%93%81"
options = Options()
driver = webdriver.Chrome(options=options)

# wordsに検索ワードを[リストで]入力
words =["airpods", "第二世代", "新品"]
url = create_index_url(words)
try:
    
        
        
    #２０００件集め終えたら一覧の取得ループも終わる
    has_next = True
    page_count = 1
    #一覧取得ループ : 
    # 次へが終わったらやめる
    while has_next :   
        print(f"一覧{page_count}ページ目取得開始...")
        url  = create_index_url(words)+ f"&page={page_count}"
#         url = create_index_url(["airpods", "第二世代","新品"])
        driver.get(url)
        time.sleep(5) #一覧は検索量が多く、5秒は待つのが吉。
        
        
#         スクロール処理(こうしないと要素が読み込まれない)
#       2019/11/17追記　
#     　静的ページに戻ったようなのでスクロールの必要がなくなった
#         height = driver.execute_script("return document.body.scrollHeight")
#     #ループ処理で少しづつ移動
#         print("ページスクロール中")
#         for x in range(1,height,5):
#             driver.execute_script("window.scrollTo(0, "+str(x)+");")
#         time.sleep(4)
#         print("ページスクロール終了")

        #一覧ページソース
        soup= BeautifulSoup(driver.page_source, "html.parser")
#         一覧ページに「該当する商品が見つかりません。検索条件を変えて、再度お試しください。」が見つかったら終わり
        if soup.select(".sc-cpmKsF"):
            print("一覧ページに「該当する商品が見つかりません」が表示されたので、取得を終了します。")
            break
#       2019/11/17追記
#        関係ない商品が表示されず、最終ページで上記「~お試しください」も表示されないので、詳細が見つからない時取得を終了する。
        elif len(soup.select(".items-box>a")) == 0:
            print("一覧ページに商品詳細がありませんので、取得を終了します。")
            break


        #一覧から商品詳細リンク95個取得し、(それぞれ<section class="items-box"> の下の aに入っている)
        # リンクタグのリストからhref属性、つまり商品詳細のurlのリストを取得
        href_list = []
        
#     2019/11/17追記
#     クラス名が.items-boxとわかりやすい名称に戻っていたので、暗号化名称箇所を削除して元に戻す

        for a in soup.select(".items-box>a"):
#         for a in soup.select(".sc-eMigcr a"):
            href_list.append(a.get("href"))
        print(f"一覧ページ{page_count}ページ目の{len(href_list)}個の詳細リンクをたどる")
        # 詳細取得ループ
        #取得したリンクたちに対してアクセスを繰り返す
        

#         属性リスト
        good_list = []
        normal_list = []
        bad_list = []
        status_list = []
        charge_list = []
        method_list = []
        region_list = []
        price_list = []
        sold_list = []
        image_count_list = []
        description_list = []

        for  i, link in enumerate(href_list):
            print(f"一覧{page_count}ページ,詳細{i+1}商品目, 累計{len(good_list)+1}商品目を取得開始...")
            detail_dictionary = get_detail_dictionary(link) #上に関数定義。取得属性をいじりたいときは上のget_detail_dictionary関数に追記。
            
            # 取得した属性での
            good_list                .append(detail_dictionary["good"])
            normal_list             .append(detail_dictionary["normal"])
            bad_list                  .append(detail_dictionary["bad"])
            status_list             .append(detail_dictionary["status"])
            charge_list            .append(detail_dictionary["charge"])
            method_list          .append(detail_dictionary["method"])
            region_list             .append(detail_dictionary["region"])
            price_list               .append(detail_dictionary["price"])
            sold_list                .append(detail_dictionary["sold"])
            image_count_list.append(detail_dictionary["image_count"])
            description_list    .append(detail_dictionary["description"])
            print(f"一覧{page_count}ページ,詳細{i+1}商品目, 累計{len(good_list)}商品目を取得完了。 {link}")
            print("-"*100)
            
            #取得数が2000に達したら、
            #または「>」がなく次ページが取得できなかったら取得終わり
#       　　2019/11/1７追記
#         詳細1ページずつ保存するので際限なく取って良い。よって制限なし
#             max_data_count = 2000
#             if len(good_list)>= max_data_count:
#                 print(f"詳細データ数が {max_data_count}に達したので取得を終了します。")
#                 has_next = False #こうすると一覧取得ループも終わる
#                 break
             #dataframe作成
        df = pd.DataFrame()
        df["good"]             = good_list
        df["normal"]          = normal_list
        df["bad"]                = bad_list
        df["status"]           = status_list
        df["charge"]           = charge_list
        df["method"]         = method_list
        df["region"]            = region_list
        df["price_list"]       = price_list
        df["sold"]                = sold_list
        df["image_count"] = image_count_list
        df["description"]     = description_list

        df.to_csv("メルカリ"+"+".join(words)+str(page_count)+"ページ目.csv")
        print("dataframe書き出し完了。")
        page_count += 1


        # 詳細取得ループ終わり。
    #一覧取得ループ終わり。

    
    
    print(df.head())
    print("全データ取得完了。お疲れ様でした。")
    
except:
    #何か失敗。エラー出力
    traceback.print_exc()
finally:
    # エラーが起きても起きなくてもブラウザを閉じる
    driver.quit()


取得を開始します
一覧1ページ目取得開始...
一覧ページ1ページ目の73個の詳細リンクをたどる
一覧1ページ,詳細1商品目, 累計1商品目を取得開始...
289 2 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1450 1 4 お待たせしました♡
大人気トイストーリーシリーズのAirpo
一覧1ページ,詳細1商品目, 累計1商品目を取得完了。 https://item.mercari.com/jp/m72823177146/?_s=U2FsdGVkX1_fjAbm2Vn04z__eV2QpXNUfzBn2JU_LNwd6G-SKK2t_zgeDafxgYSaevFCZYHd3bQ8QmFwzEefw1WMbDHv2_i5NCklzd-kanC4jDVk2jL2pIzy9GwoGrLk
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細2商品目, 累計2商品目を取得開始...
1 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1280 1 6 ★こちらはリラックマの出品ページです★
※即購入の場合、↑こ
一覧1ページ,詳細2商品目, 累計2商品目を取得完了。 https://item.mercari.com/jp/m19062691594/?_s=U2FsdGVkX1-tE7Rcmj8230EZL1HEQuSaTEhBFyNICnaSTFFLhWSdMHl3fmyCsA_9RTnEzrfHUpZNMI3e-dM79AMsDeInQj9kU297oa2rR4Ly8z-ZSq2qrPpWJiEnC-X8
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細3商品目, 累計3商品目を取得開始...
88 3 1 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 千葉県 1350 0 None ご覧頂き、ありがとうございます
Airpodsケース  


一覧1

166 2 0 目立った傷や汚れなし 送料込み(出品者負担) らくらくメルカリ便 静岡県 12000 1 9 ご覧頂きありがとうございます。
AirPods pro 購入
一覧1ページ,詳細21商品目, 累計21商品目を取得完了。 https://item.mercari.com/jp/m81736198463/?_s=U2FsdGVkX1_zc4aMQQ_J34HY8cZJ1JDLSmmRp0Ihfg6PxfmPSpk3344gT6BsekbZERQHydXfBwE-WJZXKxpnsvLhPhhl0TEJOtVYdOuyvBCZkGWQep9YUwL1tJv1jDrv
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細22商品目, 累計22商品目を取得開始...
119 3 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 愛媛県 1500 1 2 ありがとうございます。

☆販売商品
・新品 Apple A
一覧1ページ,詳細22商品目, 累計22商品目を取得完了。 https://item.mercari.com/jp/m57442925348/?_s=U2FsdGVkX1_S7QFRGgcOGYWeoEtnVe8U7bsvlvcMTQgFAH3nSTJQI-ToIIZAlcr5Z2pvGdORCWkCxApoPj921J_HnkY4VML_jBqyRk86FP-bEAAA2JCBG-uRcpNHE9m1
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細23商品目, 累計23商品目を取得開始...
19 1 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 999 1 7 モンスターズインクのAirpodsケース!
ケースを付けたま
一覧1ページ,詳細23商品目, 累計23商品目を取得完了。 https://item.mercari.com/jp/m75085103986/?_s

89 0 0 未使用に近い 送料込み(出品者負担) らくらくメルカリ便 愛知県 17800 1 4 10回の室内使用のみ、新型が欲しいため出品します、
バッテリ
一覧1ページ,詳細41商品目, 累計41商品目を取得完了。 https://item.mercari.com/jp/m88802939955/?_s=U2FsdGVkX1843iVAHzj6Hb5ARSCjDYJjUgmFKcMPhz4Dl8t5fOn11LWvDnDx6iuOybM417VGQpkPsE7MnObmIjILpj_lRRa4bA6kXRXeaZL9SpBIDd02D6bATYfc24h8
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細42商品目, 累計42商品目を取得開始...
12 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1450 0 5 【こちらはバズ・ライトイヤーのページです】
※※※即購入の場
一覧1ページ,詳細42商品目, 累計42商品目を取得完了。 https://item.mercari.com/jp/m24718855233/?_s=U2FsdGVkX1_uqj534O_mNI6NK0NLm7EzoB-0Y3Gc5Qa_0ISMcLkMjVRoSyl3bo3sljJTrBs4C-jIulPF7XIT7OpmN59TRAyX7O6Rh0b1GL6dGreHL3wiSjaTkeUKmoFx
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細43商品目, 累計43商品目を取得開始...
585 14 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1450 1 6 大人気トイストーリーのAirPodsケースです♪
しっかりし
一覧1ページ,詳細43商品目, 累計43商品目を取得完了。 https://item.mercari.com/jp/m15351504903/?_s=U2

38 0 0 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 大阪府 6800 1 None Apple 第2世代(MV7N2J/A)のAirPods、正
一覧1ページ,詳細61商品目, 累計61商品目を取得完了。 https://item.mercari.com/jp/m87924110238/?_s=U2FsdGVkX19QqZxU7L3zx7XbmrP-s4pyW-Wo30r_9ABA88BOAh6P26cVkmbBQrrvmNz1aeuSL82gw3j8T6MDtDkuvaRu4zQBWOgpohubOs2BN8z1WU--9mEH25FQoWlQ
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細62商品目, 累計62商品目を取得開始...
38 0 0 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 大阪府 6800 1 None Apple 第2世代(MV7N2J/A)のAirPods、正
一覧1ページ,詳細62商品目, 累計62商品目を取得完了。 https://item.mercari.com/jp/m13631869595/?_s=U2FsdGVkX18UmLxbqFKb_aLzjsuuyE67t91d5HWPLUosyI3AQeUSAO5J8u_QToegu9K7W73ZjLXMYMhO1vj8UY_kEcN7IEeBdMQlhhQ65sCCZzq7aq6JRrN0HtdEgn2_
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細63商品目, 累計63商品目を取得開始...
13 0 0 目立った傷や汚れなし 送料込み(出品者負担) らくらくメルカリ便 東京都 6222 1 3 L片耳、左のみの商品です。
動作確認済み・消毒済み
※箱、ラ
一覧1ページ,詳細63商品目, 累計63商品目を取得完了。 https://item.mercari.com/jp/m29531528676/?_s=U

80 1 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 宮城県 18650 1 5 【保証書付・新品・未使用】AirPods第二世代！ 

20
一覧2ページ,詳細8商品目, 累計8商品目を取得完了。 https://item.mercari.com/jp/m95176033172/?_s=U2FsdGVkX19XTvjF5JVIm7p2vKyqha28sV-1ssu91y2AJ6cQRxuxJtAGGBhJ-6Ml4TTdLW0EUYlqfkufr6Mxw006SfUbDDTvbOtErjBRpMFwlv7kx5OUIhg5C16ZdP_X
----------------------------------------------------------------------------------------------------
一覧2ページ,詳細9商品目, 累計9商品目を取得開始...
351 8 2 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 埼玉県 20500 1 3 Apple製
最高級品ワイヤレスイヤホン
定価25,080円
一覧2ページ,詳細9商品目, 累計9商品目を取得完了。 https://item.mercari.com/jp/m84919311076/?_s=U2FsdGVkX19Ql6wIqJJTOpVqwX4s6QFAI_s9CYZAoZLjVG2Pc0S0YHLc-AEapfAJwFy9Q5emjpPMozfuJ-mTpfDh1hUc_cCeRtRhCsZQ9ExdqkdYCyIP8vLC4JoDVivV
----------------------------------------------------------------------------------------------------
一覧2ページ,詳細10商品目, 累計10商品目を取得開始...
289 2 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1120 0 4 ディズニーくまのプーさん＆ピグレットAirPodsケース♡

一覧2ページ,詳細10商品目, 累計10商品目を取得完了。 https://item.mercari.com/jp/m86704489015/?_s=U2FsdGVkX1

...

87 0 1 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1120 0 7 【こちらはプーさんの出品ページです。】

上記商品でお間違え
一覧3ページ,詳細20商品目, 累計20商品目を取得完了。 https://item.mercari.com/jp/m62284906420/?_s=U2FsdGVkX1_N4ALUsQjuC4Ups0p2VdJPC9CDDZBiegtqk3h96Z0_Ul_arMAmUYhqQlEgqMP1SsK_U12bqgQpJTc2jRRvc-XiwKRw0KH0Cn0acTiwKpYV-TgEybn6lCG9
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細21商品目, 累計21商品目を取得開始...
119 3 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 愛媛県 999 1 2 ありがとうございます。

☆販売商品
・新品 Apple A
一覧3ページ,詳細21商品目, 累計21商品目を取得完了。 https://item.mercari.com/jp/m81980209060/?_s=U2FsdGVkX19gUriLIyPLljN3EZW0jNnPos-_Nw8_LSZgs75DvMvMjS3aurUxilKJ2JKN2hU7tiguee7qpPbkBuP7DRunQPNiWFJuGD37YpQX3UZeAQP1JIpML528l9BL
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細22商品目, 累計22商品目を取得開始...
141 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 27299 0 6 アップル 純正品 エアーポッズ Apple AirPods 
一覧3ページ,詳細22商品目, 累計22商品目を取得完了。 https://item.mercari.com/jp/m52579879009/?_s=U2F

49 0 0 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 神奈川県 599 1 3 【新品未使用】

即購入OKです。

AirPodsケースで
一覧3ページ,詳細40商品目, 累計40商品目を取得完了。 https://item.mercari.com/jp/m23546196234/?_s=U2FsdGVkX1_r6w5ploJdzefYMUp1EQ88NIYIj5KsaUk5IzHvPbYboXgznvhB2mGBqOXJVdOz26qEwsg7GQPoM_hKcc_SI9sdmuKwUcrefJzLhZTNoSE1E-iWEBTAYkOj
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細41商品目, 累計41商品目を取得開始...
14 0 3 新品、未使用 送料込み(出品者負担) 未定 大阪府 2777 0 4 ✨新品未使用箱付きの値段です！！✨
他にはありません！！最安
一覧3ページ,詳細41商品目, 累計41商品目を取得完了。 https://item.mercari.com/jp/m23159227026/?_s=U2FsdGVkX186wsvom6QT3-0I3h2ic6f1s8OrYJpbdgPtrZqQ-AeFWlUQSsGHYoxbW9ADvN5iotfxFaa-5jYquQy-EQl-672-nUQ0TTVkKhKkGnEoqDvt4lD8aHD_sfhq
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細42商品目, 累計42商品目を取得開始...
14 0 3 新品、未使用 送料込み(出品者負担) 未定 大阪府 2777 0 4 ✨新品未使用箱付きの値段です！！✨
他にはありません！！最安
一覧3ページ,詳細42商品目, 累計42商品目を取得完了。 https://item.mercari.com/jp/m92506375610/?_s=U2FsdGVkX1-zWpCxUjFbNMZ0v

596 4 0 未使用に近い 送料込み(出品者負担) らくらくメルカリ便 東京都 17399 1 4 Apple AirPods with Charging Ca
一覧3ページ,詳細60商品目, 累計60商品目を取得完了。 https://item.mercari.com/jp/m60073570107/?_s=U2FsdGVkX1-GPoRjKhM1QCaR0uR8amqi4W2E650TVK5l8QRU0nKB1SgBheu43mxWZ9kDHKhmM8mJ9aCUMfBbm15IOpNJex-A2u1niIOTQm0FNMIcVvjjkkG_rg69tC3d
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細61商品目, 累計61商品目を取得開始...
23 0 0 新品、未使用 送料込み(出品者負担) 未定 奈良県 490 1 6 商品説明


AirPods第二世代と第一世代両方にぴったり
一覧3ページ,詳細61商品目, 累計61商品目を取得完了。 https://item.mercari.com/jp/m13571090951/?_s=U2FsdGVkX19tNICcf_8s0OQjgof-6CixRFyUDB53CLYLW8h4rQQMuywRCdOhhOAawRdzno97RGiqATLRMiEstkclgkwExGwsWrJMgMFWahGNPYmv1jHW_DCTFzLHfh7H
----------------------------------------------------------------------------------------------------
一覧3ページ,詳細62商品目, 累計62商品目を取得開始...
211 6 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 栃木県 18900 1 2 Apple AirPods with Charging Ca
一覧3ページ,詳細62商品目, 累計62商品目を取得完了。 https://item.mercari.com/jp/m56849525472/?_s=U2FsdGVkX19qzt7

13 0 0 やや傷や汚れあり 送料込み(出品者負担) らくらくメルカリ便 東京都 3555 1 5 AirPodsをご覧いただきありがとうございます。
 第二世
一覧4ページ,詳細18商品目, 累計18商品目を取得完了。 https://item.mercari.com/jp/m25557732577/?_s=U2FsdGVkX19pKv5K081hqXX3x7soo9B20KmQ8iQEHmL26ccGv_sV4f0Kr8-K2UZ1mtPWcVnUmslONdYlgtbHs4GQsZSLnfj-YixHkQ9TY9Pu2PZ4AYhX5m0MD1O3L-us
----------------------------------------------------------------------------------------------------
一覧4ページ,詳細19商品目, 累計19商品目を取得開始...


Traceback (most recent call last):
  File "<ipython-input-6-b50a64f130f5>", line 85, in <module>
    detail_dictionary = get_detail_dictionary(link) #上に関数定義。取得属性をいじりたいときは上のget_detail_dictionary関数に追記。
  File "<ipython-input-4-9cf641528423>", line 4, in get_detail_dictionary
    driver.get(link)
  File "/Users/aki/ticket_tenbai/my_env/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "/Users/aki/ticket_tenbai/my_env/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/Users/aki/ticket_tenbai/my_env/lib/python3.7/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=78.0.3904.97)



In [7]:
# # 商品名とページ番号nを与えると、商品一覧nページ目にある商品詳細情報を取得して一覧に返す
# # スクロールは手動で行うこととする


# # options = Options()
# # driver = webdriver.Chrome(options=options)

# words =["airpods", "第二世代", "新品"]
# page = 1
# url = create_index_url(words)
# try:
#     good_list = []
#     normal_list = []
#     bad_list = []
#     status_list = []
#     charge_list = []
#     method_list = []
#     region_list = []
#     price_list = []
#     sold_list = []
#     image_count_list = []
#     description_list = []
        
    
#     has_next = True
#     page_count = 1
#     #一覧取得ループ : 

#     print(f"一覧{page_count}ページ目取得開始...")
#     url = create_index_url(words)
#     driver.get(url)
# #     from IPython.core.debugger import Pdb; Pdb().set_trace()
#     #ここで手動スクロールする→いやだ。自動スクロールする
    
#     height = driver.execute_script("return document.body.scrollHeight")
#     #ループ処理で少しづつ移動
#     for x in range(1,height,5):
#         driver.execute_script("window.scrollTo(0, "+str(x)+");")
#     time.sleep(5) #一覧は検索量が多く、5秒は待つのが吉。
#     soup= BeautifulSoup(driver.page_source, "html.parser")

#     #一覧から商品詳細リンク95個取得し、(それぞれ<section class="items-box"> の下の aに入っている)
#     # リンクタグのリストからhref属性、つまり商品詳細のurlのリストを取得
#     href_list = []
# #         for a in soup.select(".items-box>a"):
    
#     for a in soup.select(".sc-eMigcr a"):
#         href_list.append(a.get("href"))
#     print(f"{len(href_list)}個のリンクをたどる")
#     for  i, link in enumerate(href_list):
#         print(f"一覧{page_count}ページ,詳細{i}商品目, 累計{len(good_list)+1}商品目を取得開始...")
#         detail_dictionary = get_detail_dictionary(link) #上に関数定義。取得属性をいじりたいときは上のget_detail_dictionary関数に追記。
            
#         # 取得した属性での
#         good_list                .append(detail_dictionary["good"])
#         normal_list             .append(detail_dictionary["normal"])
#         bad_list                  .append(detail_dictionary["bad"])
#         status_list             .append(detail_dictionary["status"])
#         charge_list            .append(detail_dictionary["charge"])
#         method_list          .append(detail_dictionary["method"])
#         region_list             .append(detail_dictionary["region"])
#         price_list               .append(detail_dictionary["price"])
#         sold_list                .append(detail_dictionary["sold"])
#         image_count_list.append(detail_dictionary["image_count"])
#         description_list    .append(detail_dictionary["description"])
#         print(f"一覧{page_count}ページ,詳細{i}商品目, 累計{len(good_list)}商品目を取得完了。 {link}")
#         print("-"*100)
            

    
#      #dataframe作成
#     df = pd.DataFrame()
#     df["good"]             = good_list
#     df["normal"]          = normal_list
#     df["bad"]                = bad_list
#     df["status"]           = status_list
#     df["charge"]           = charge_list
#     df["method"]         = method_list
#     df["region"]            = region_list
#     df["price_list"]       = price_list
#     df["sold"]                = sold_list
#     df["image_count"] = image_count_list
#     df["description"]     = description_list
    
#     os.makedirs("+".join(words),exist_ok=True)
#     df.to_csv("+".join(words)+f"{page}ページ.csv")
#     print("dataframe書き出し完了。")
    
#     print(df.head())
#     print("全データ取得完了。お疲れ様でした。")
    
# except:
#     #何か失敗。エラー出力
#     traceback.print_exc()
# finally:
#     # エラーが起きても起きなくてもブラウザを閉じる
#     driver.quit()


In [16]:
#　以下はテストなので実行する必要なし。

#　成功: テスト: Google検索ぺーじのaタグ取得()
# [Python \+ Selenium \+ ChromeでWebスクレイピングをやってみる \- Qiita](https://qiita.com/cl4nny_k/items/096ddae866a4886710e9)

# 成功: テスト: driver表示ページからSoupオブジェクトを取得
# driver.page_sourceに入っている
# options = Options()
# ヘッドレスモードで実行する場合
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

# try:
#     driver.get("https://www.google.co.jp/")
#     # 簡易的にJSが評価されるまで秒数で待つ
#     time.sleep(5)

#     #検索結果のページのHTMLをBeautifulSoupに流し込む
#     soup = BeautifulSoup(driver.page_source, "html.parser")
#     print(soup.select("a")[0])
# #     # aタグを抽出
# #     elem_list = driver.find_elements_by_tag_name("a")
# #     for elem in elem_list:
# #         # attributeの中からhrefを抽出して出力
# #         url = elem.get_attribute("href")
# #         print(url)

# except:
#     traceback.print_exc()
# finally:
#     # エラーが起きても起きなくてもブラウザを閉じる
#     driver.quit()

<a class="gb_e" data-pid="23" href="https://mail.google.com/mail/?tab=wm&amp;ogbl">Gmail</a>


In [37]:
#　レシーバって間を空けていいの？
# →　良い。
w = []
w   .   append("hello")
w


['hello']

In [16]:
d = pd.read_csv("airpods+第二世代+新品.csv",index_col=0)

#テスト：NaNがないか
# 成功：ない
# d.isnull().max()
d

,good,normal,bad,status,charge,method,region,price_list,sold,image_count,description
0,14,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,1480,0,8.0,大人気ダッフィーフレンズのステラルーの\nAirpodsケース\nケースを付けたまま充電もで...
1,192,6,1,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,山口県,19999,0,2.0,★即日発送（早ければ数時間以内に即発送します！）\n★匿名配送\n\n\n安心の配送事故補償...
2,15,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),奈良県,1600,1,4.0,新品未開封\n2019年新型 Airpods第2世代 Airpods2 新型Airpo...
3,505,2,0,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,東京都,18999,1,3.0,Apple MV7N2J/A Airpods with Charging Case 第2世代...
4,3,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,1260,0,6.0,大人気トイストーリーのリトルグリーンメン♪\nこちらはAirpodsケースになります(*ˊᵕ...
5,1239,2,0,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,大阪府,18500,1,3.0,AirPods with Charging Case 第二世代\n製品番号：MV7N2J/...
6,47,1,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),埼玉県,1000,1,NaN,AirPodsケース\n第一世代、第二世代に適用。\n新品未使用ですが箱などはありません。\...
7,276,2,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,999,0,7.0,※※こちらは訳ありB級品です※※\n\n\n ☆訳あり部分について☆\n\n詳細は...
8,674,10,2,新品、未使用,送料込み(出品者負担),未定,東京都,450,1,7.0,===================\n新品未使用\n全国送料無料 \nコ...
9,907,20,12,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,北海道,19700,0,3.0,Apple MV7N2J/A Airpods with Charging Case \nAi...


In [ ]:
# テスト　breakの有効なスコープ

a = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"]]
abc = True
count = 0
while abc :
    arr = a[count]
    for i in arr:
        if i =="e":
            print("終了します。")
            break
        print(i)
    count += 1
    if count >=3:
        break
